<a href="https://colab.research.google.com/github/MsAsterisk1/ERM_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: NVIDIA GeForce RTX 2060 (UUID: GPU-100f20af-1b36-5adc-0795-f06b020c050b)


In [3]:
# !git clone https://github.com/MsAsterisk1/ERM_GDRO.git
# %cd /content/ERM_GDRO

In [4]:
# !git pull

In [5]:
# !pip install transformers

In [ ]:
import pandas as pd
from utils.process_data_utils import  get_CivilComments_DataLoaders, get_CivilComments_Datasets, get_CivilComments_df
from loss import ERMLoss, GDROLoss
from train_eval import train, evaluate, train_epochs
import torch
from models import BertClassifier

Now lets try getting Bert working

In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [33]:
df = get_CivilComments_df()

In [ ]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [ ]:
train,cv,test=  get_CivilComments_DataLoaders(datasets=datasets)

This is how we would initialize model

In [12]:
model = BertClassifier().to(DEVICE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, train, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=8)


Epoch 1 / 5
Epoch 2 / 5
Epoch 3 / 5
Epoch 4 / 5
Epoch 5 / 5


In [14]:
torch.save(model.state_dict(), './CC_ERM.wt')


In [18]:
model = BertClassifier().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, train, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=8)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1 / 5
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False], device='cuda:0')
tensor([False, False, False, False, False, False,  True, False, False,  True,
        False,  True, False, False, False, False], device='cuda:0')
tensor([False, False, False, False, False, False, False, False, False, False,
         True,  True, False, False, False, False], device='cuda:0')
tensor([False, False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False], device='cuda:0')
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False], device='cuda:0')
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False], device='cuda:0')
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False

RuntimeError: CUDA out of memory. Tried to allocate 66.00 MiB (GPU 0; 6.00 GiB total capacity; 5.10 GiB already allocated; 0 bytes free; 5.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.save(model.state_dict(), './CC_GDRO.wt')
